# load data

In [48]:
import json
import sqlite3
import pandas as pd
import re
from urllib.parse import urlparse
import requests

In [2]:
from datetime import date
from datetime import timedelta

## load tweets older then two weeks

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/tweetfeed")
    
from data import load_tweets


In [5]:
# data keep on comming - to work on a static dataset I save it to pickle 

In [6]:
# df_tweets = load_tweets("../home.db", days=0)
# df_tweets = df_tweets[df_tweets.retweeted_status == "N/A"] # remove RT
# df_tweets.to_pickle("shorturl.pkl")

In [7]:
df_tweets = pd.read_pickle("shorturl.pkl"); df_tweets.shape

(74869, 9)

# utils

In [8]:
df_tweets.tail()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id
95305,1359964994532614144,2178012643,"It’s one thing to ban violent crimes, universa...",2021-02-11T20:38:16+00:00,en,N/A,N/A,0,1359963436117938178
95306,1359965511396675585,15626406,(It would plausibly reduce corn subsidies - le...,2021-02-11T20:40:19+00:00,en,N/A,N/A,0,1359964356541181952
95307,1359965679110287360,259034658,Winter dopamine famine.,2021-02-11T20:40:59+00:00,de,N/A,N/A,0,N/A
95308,1359965833502494722,817386,Google adds some of its Pixel-exclusive photo ...,2021-02-11T20:41:36+00:00,en,N/A,N/A,0,N/A
95309,1359965919028547587,1416500532,"And just like that, a migraine emerges,,,",2021-02-11T20:41:56+00:00,en,N/A,N/A,0,N/A


In [9]:
from requests_html import HTMLSession
url = "https://lukecarneal.substack.com/p/why-are-some-organic-farmers-turning"
print(HTMLSession().get(url).html.find('title', first=True).text)

Why Are Some Organic Farmers Turning to Reactionary Politics? - The Farm Worker Bulletin


In [10]:
def grab_title(url):
    try:
        title = HTMLSession().get(url).html.find('title', first=True).text
        return title
    except Exception as ex:
        return 0

In [11]:
url = "https://buff.ly/2T4jips"
grab_title(url)

'3 Tiny Mental Habits I Practice Every Morning – In Less Than 5 Minutes'

In [29]:
from data import remove_tw_urls, rem_short_links, find_url, get_domain, remove_empty_str

def find_news(df, news_domains_list):
    df = df.copy()
    df["clean_text"] = df["full_text"].apply(remove_tw_urls).apply(rem_short_links)
    df["clean_text"] = df["clean_text"].apply(rem_short_links)
    df["urls"] = df["clean_text"].apply(find_url)
    df.drop(["clean_text"], axis=1, inplace=True)
    df["domains"] = df.urls.apply(lambda x: [get_domain(d) for d in x])
    df["domains"] = df.domains.apply(remove_empty_str)
    # df.drop(["urls"], axis=1, inplace=True)

    new_columns_list = []
    max_nr_dom = df.domains.str.len().max()
    for i in range(max_nr_dom):
        new_columns_list.append(f"domain{i+1}")
    df.reset_index(drop=True, inplace=True)
    df[new_columns_list] = pd.DataFrame(df.domains.tolist())

    for col in new_columns_list:
        df[col] = df[col].isin(news_domains_list)

    #     df.drop(["domains"], axis=1, inplace=True)

    df["contains_news"] = df[new_columns_list].sum(axis=1)
    df["contains_news"] = df.contains_news.apply(lambda x: x if x == 0 else 1)
    df.drop(new_columns_list, axis=1, inplace=True)

    return df

In [30]:
def drop_contains(df, column_name, word_list):
    for string in word_list:
        df["lower"] = df["full_text"].str.lower()
        df = df[df["lower"].str.contains(string)]
        df.drop(["lower"], axis=1, inplace=True)
    return df

# searching for not-expanded news urls

In [31]:
dfz = df_tweets.copy()

In [32]:
with open("news_domains.txt", "r") as f:
    news_domains = json.loads(f.read())

In [33]:
dfz = find_news(dfz, news_domains)

In [34]:
dfz["has_domain"] = dfz.domains.apply(lambda x: len(x))

In [35]:
dfz.head()

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,urls,domains,contains_news,has_domain
0,5596520,713143,soo hungry need to find my wife and head to pf...,2007-02-20T02:43:19+00:00,en,N/A,N/A,0,N/A,[],[],0,0
1,154674522,7193842,Hofstadter’s Law: A task always takes longer t...,2007-07-17T20:45:46+00:00,en,N/A,N/A,0,N/A,[],[],0,0
2,412145582,4829901,Adobe Updater must update itself before instal...,2007-11-13T21:45:35+00:00,en,N/A,N/A,0,N/A,[],[],0,0
3,922321981,16298441,no,2008-09-15T17:25:20+00:00,und,N/A,N/A,0,N/A,[],[],0,0
4,2627602600,21454322,"Went on a USO trip to Guantanamo Bay, Cuba a f...",2009-07-14T05:15:27+00:00,en,N/A,N/A,0,N/A,[],[],0,0


In [36]:
# take tweets that have domains
dfz = dfz[dfz.has_domain > 0]
dfz.shape

(19235, 13)

In [37]:
# let's grab only those WITHOUT news
dfz = dfz[dfz.contains_news == 0]
dfz.shape

(15120, 13)

In [38]:
# dfz.reset_index(drop=True, inplace=True)

In [39]:
dfz.iloc[452]


id                                                     1330742961177391107
user                                                              33521530
full_text                for those worried about app compatibility, @_p...
created_at                                       2020-11-23T05:20:20+00:00
lang                                                                    en
retweeted_status                                                       N/A
quoted_status                                                          N/A
is_quote_status                                                          0
in_reply_to_status_id                                  1330432426649202698
urls                       [https://isapplesiliconready.com/for/developer]
domains                                          [isapplesiliconready.com]
contains_news                                                            0
has_domain                                                               1
Name: 2171, dtype: object

In [ ]:
def grab_title(url):
    try:
        title = HTMLSession().get(url).html.find('title', first=True).text
        return title
    except Exception as ex:
        return 0

In [47]:
dfz.urls[:10]

7                   [http://bitcoincharts.com/markets/]
25                [http://stanford.edu/~zdar/week1.pdf]
34    [http://imdb.com/rg/an_share/title/title/tt005...
35    [http://bandedessinee.blog.lemonde.fr/2014/06/...
39    [http://link.springer.com/article/10.1007/s001...
52    [http://www.sankeisquare.com/event/kanjicontes...
55                           [http://gifthefeeling.com]
56          [https://archive.org/details/malwaremuseum]
59    [https://www.reddit.com/r/MarioMaker/comments/...
65                                  [https://kite.com/]
Name: urls, dtype: object

In [52]:
import warnings; warnings.simplefilter('ignore')
# there will be InsecureRequestWarning

In [56]:
# LocationParseError: Failed to parse: http://fast.ai’s 177

CPU times: user 3.23 s, sys: 25.9 ms, total: 3.26 s
Wall time: 3.28 s


In [137]:
import json

# short_url =[]
with open("short_url.json") as json_file:
    short_url = json.load(json_file)
urls_list = pd.DataFrame(short_url).url.tolist()

x = 2701
for c, i in enumerate(dfz.urls[x:]):
    print(c + x, i)
    if (c + x + 1) % 100 == 0:
        print(f"{(c+1)} / {len(dfz.urls)}")
    for link in i:
        print(c+x)
        if link in urls_list:
            pass
        
        else:
            try:
                char_to_rem = "',)\"!" 
                for char in char_to_rem:
                    link = link.replace(char, "")
                if link.split(".")[-1] == "pdf":
                    short_url.append(
                        {
                            "url": link,
                            "links": i,
                            "idx": c+x,
                            "error": "pdf",
                        }
                    )
                else:
                    domain = get_domain(link)
                    response = HTMLSession().get(link, verify=False)
                    e_link = response.url
                    response.html.find('title', first=True).text
                    dom_expanded = get_domain(e_link)
                    short_url.append(
                        {
                            "url": link,
                            "title": response.html.find('title', first=True).text,
                            "short_url": domain,
                            "long_url": dom_expanded,
                            "is_news": (dom_expanded in news_domains),
                            "diff_url": (dom_expanded != domain),
                        }
                    )
            except Exception as ex:
                short_url.append(
                    {
                        "url": link,
                        "links": i,
                        "idx": c+x,
                        "error": str(ex),
                    }
                )
    with open("short_url.json", "w") as file:
        file.write(json.dumps(short_url, indent=4))

2672 ['https://tinyurl.com/y39e2vhc']
2672
2673 ['https://youtu.be/R07CVhWbAXc']
2673
2674 ['https://www.cbinsights.com/research-unicorn-companies']
2674
2675 ['https://www.harvardartmuseums.org/collections/object/198036']
2675
2676 ['https://brainpint.com/']
2676
2677 ['https://www.harvardartmuseums.org/collections/object/306083']
2677
2678 ['http://nomadlist.com/@replay']
2678
2679 ['https://slideslive.com/38938169/magenta-empowering-creative-agency-with-machine-learning']
2679
2680 ['https://github.com/org-formation/org-formation-cli/issues/131#issuecomment-744051926']
2680
2681 ['https://www.harvardartmuseums.org/collections/object/310336']
2681
2682 ['https://deepai.org/publication/learning-video-instance-segmentation-with-recurrent-graph-neural-networks']
2682
2683 ['https://en.wikipedia.org/wiki/Paradox_of_tolerance']
2683
2684 ['https://youtu.be/QIUrd327tOQ']
2684
2685 ['https://extranewsfeed.com/tolerance-is-not-a-moral-precept-1af7007d6376']
2685
2686 ['https://ophi.org.uk/wp

KeyboardInterrupt: 

In [112]:
# short_url =[]
with open("short_url.json") as json_file:
    short_url = json.load(json_file)
dff = pd.DataFrame(short_url)

In [ ]:
song.replace('o', 'e')

In [120]:
char_to_rem = "',)\"!" 

In [128]:
asdf = dff.iloc[2061].url

In [129]:
%%time
for i in char_to_rem:
    asdf = asdf.replace(i, "")
print(asdf)

http://jina.ai
CPU times: user 189 µs, sys: 0 ns, total: 189 µs
Wall time: 159 µs


In [115]:
dff[~dff.idx.isna()]

,url,title,short_url,long_url,is_news,diff_url,idx,error,links
199,http://fast.ai’s,NaN,NaN,NaN,NaN,NaN,178.0,Failed to parse: http://fast.ai’s,NaN
648,http://Gather.Town!,NaN,NaN,NaN,NaN,NaN,585.0,"HTTPConnectionPool(host='gather.town!', port=8...",[http://Gather.Town!]
834,http://animalbehaviourcollective.org),NaN,NaN,NaN,NaN,NaN,759.0,HTTPConnectionPool(host='animalbehaviourcollec...,"[http://animalbehaviourcollective.org), https:..."
964,"http://shop.boox.com,",NaN,NaN,NaN,NaN,NaN,872.0,"HTTPConnectionPool(host='shop.boox.com,', port...","[http://shop.boox.com,]"
1185,http://www.fdle.state.fl.us/News/2020/December...,NaN,NaN,NaN,NaN,NaN,1064.0,HTTPConnectionPool(host='www.fdle.state.fl.us'...,[http://www.fdle.state.fl.us/News/2020/Decembe...
1323,"http://qrmenucreator.com,",NaN,NaN,NaN,NaN,NaN,1183.0,"HTTPConnectionPool(host='qrmenucreator.com,', ...","[http://qrmenucreator.com,, http://qrmn.co, ht..."
1463,http://epns.io!,NaN,NaN,NaN,NaN,NaN,1307.0,"HTTPConnectionPool(host='epns.io!', port=80): ...",[http://epns.io!]
1864,httpJunkie,NaN,NaN,NaN,NaN,NaN,1653.0,Invalid URL 'httpJunkie': No schema supplied. ...,"[httpJunkie, https://www.nocsdegree.com/this-s..."
1942,http://c3.cloud,NaN,NaN,NaN,NaN,NaN,1728.0,"HTTPConnectionPool(host='c3.cloud', port=80): ...","[http://c3.ai, http://c3.cloud, http://c3.app,..."
2015,"https://stop.lying.cloud,",NaN,NaN,NaN,NaN,NaN,1785.0,"HTTPSConnectionPool(host='stop.lying.cloud,', ...","[https://stop.lying.cloud,]"


In [117]:
dff.iloc[2061].url

'http://jina.ai"'

In [76]:
with open('short_url.json') as json_file:
    short_url = json.load(json_file)

In [77]:
short_urls_list = pd.DataFrame(short_url)

In [78]:
short_urls_list

,url,title,short_url,long_url,is_news,diff_url
0,http://bitcoincharts.com/markets/,0,bitcoincharts.com,bitcoincharts.com,False,False
1,http://stanford.edu/~zdar/week1.pdf,Object not found! : Stanford University,stanford.edu,stanford.edu,False,False
2,http://imdb.com/rg/an_share/title/title/tt0055...,Confessions of an Opium Eater (1962) - IMDb,imdb.com,imdb.com,False,False
3,http://bandedessinee.blog.lemonde.fr/2014/06/0...,Robert Capa entre noir et blanc et clair-obscu...,bandedessinee.blog.lemonde.fr,lemonde.fr,False,True
4,http://link.springer.com/article/10.1007/s0012...,Reversal of type 2 diabetes: normalisation of ...,springer.com,springer.com,False,False
...,...,...,...,...,...,...
195,http://ethics.fast.ai/,Practical Data Ethics | Data ethics,ethics.fast.ai,ethics.fast.ai,False,False
196,https://engineering.fb.com/production-engineer...,Async: Driving efficiency and developer produc...,fb.com,fb.com,False,False
197,https://eugeneyan.com/writing/nlp-supervised-l...,NLP for Supervised Learning - A Brief Survey,eugeneyan.com,eugeneyan.com,False,False
198,https://www.fast.ai/2020/08/21/fastai2-launch/,"fast.ai releases new deep learning course, fou...",fast.ai,fast.ai,False,False


In [70]:
urls_list = pd.DataFrame(short_url).url.tolist()

In [40]:
short_urls_list = pd.DataFrame(short_url).short_url.tolist()

In [41]:
%%time
dfz["lolz"] = dfz.urls.apply(grab_title)
dfz["lolz"]

['bandedessinee.blog.lemonde.fr',
 'gifthefeeling.com',
 'kite.com',
 'ow.ly',
 'youtu.be',
 'differantly.com',
 'youtu.be',
 'buff.ly',
 'www2.philly.com',
 'bit.ly']

In [97]:
new_columns_list = []
max_nr_dom = dfz.urls.str.len().max()
for i in range(max_nr_dom):
    new_columns_list.append(f"url{i+1}")
dfz[new_columns_list] = pd.DataFrame(dfz.urls.tolist())

In [98]:
dfz

,id,user,full_text,created_at,lang,retweeted_status,quoted_status,is_quote_status,in_reply_to_status_id,urls,...,domain3,domain4,domain5,domain6,url1,url2,url3,url4,url5,url6
7,35466982635601920,15012642,#Bitcoin reaches parity with the US Dollar! h...,2011-02-09T22:36:02+00:00,en,N/A,N/A,0,N/A,[http://bitcoincharts.com/markets/],...,None,None,None,None,https://archive.org/details/malwaremuseum,None,None,None,None,None
25,451159870306549761,159169312,Course slides from @Stanford and @stanfordsyms...,2014-04-02T00:51:09+00:00,en,N/A,N/A,0,N/A,[http://stanford.edu/~zdar/week1.pdf],...,None,None,None,None,https://archive.org/download/spontaneous_assem...,None,None,None,None,None
34,563101166771650561,1460035021,"Watched Vincent Price in ""Confessions of an Op...",2015-02-04T22:25:54+00:00,en,N/A,N/A,0,N/A,[http://imdb.com/rg/an_share/title/title/tt005...,...,None,None,None,None,https://youtu.be/o_4EX4dPppA,None,None,None,None,None
35,574518676575162369,68132773,Robert Capa's Omaha Beach by Dominique Bertail...,2015-03-08T10:35:01+00:00,en,N/A,N/A,0,N/A,[http://bandedessinee.blog.lemonde.fr/2014/06/...,...,None,None,None,None,https://github.com/s0md3v/AwesomeXSS/blob/mast...,https://github.com/s0md3v/AwesomeXSS/blob/mast...,None,None,None,None
39,627427921624481792,794010396,600cals per day for 7 days reverses T2 diabete...,2015-08-01T10:37:28+00:00,en,N/A,N/A,0,N/A,[http://link.springer.com/article/10.1007/s001...,...,None,None,None,None,https://www.youtube.com/watch?v=UTy8g93Trsk,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74845,1359958734471106566,39547749,I always like to hear about knowledge engineer...,2021-02-11T20:13:23+00:00,en,N/A,N/A,0,N/A,[https://vimeo.com/79399404],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74847,1359959459578187779,1285190006970241024,@deepfates @mitdasein hat tip to @SamoBurja bu...,2021-02-11T20:16:16+00:00,en,N/A,N/A,0,1359959050536906752,[https://en.wikipedia.org/wiki/Lycurgus_Cup],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74854,1359962431884152834,1149142678715527169,#TBT to 2016 when @SnorkelAI team and research...,2021-02-11T20:28:05+00:00,en,N/A,N/A,0,N/A,[https://arxiv.org/pdf/1605.07723.pdf],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74855,1359962608917311488,16984977,New post: Lessons from Writing a Million Words...,2021-02-11T20:28:47+00:00,en,N/A,N/A,0,N/A,[http://deliprao.com/archives/401],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
%%time
dfz[:50].url1.apply(grab_title)

CPU times: user 4.23 s, sys: 26.6 ms, total: 4.26 s
Wall time: 48 s


7      The Malware Museum : Free Software : Free Down...
25              spontaneous_assembly_2 directory listing
34     "The Hard Parts of Open Source" by Evan Czapli...
35     AwesomeXSS/polyglot.png at master · s0md3v/Awe...
39     BEING AN ENTREPRENEUR | Gary Vaynerchuk With L...
52     So, You want to Contribute to Cpython : Gather...
55       Get your work recognized: write a brag document
56                              Logowanie – Konta Google
59                                     Gang Fit (Part 1)
65          Marcia Bjornerud: Timefulness - The Long Now
87     [2002.01080] Bridging the Gap: Providing Post-...
102    How 3 Michelin Star Sushi Legend Masa Cooks at...
103    A Quick 1-Min Twitter Audit to Find Your Real ...
112                                      Taiwan Can Help
117    How To Design A Good API and Why it Matters - ...
121                   ‎We Be Imagining on Apple Podcasts
143    [2006.14779] Does the Whole Exceed its Parts? ...
153              Twitter Demetr

In [49]:
# checked_url = pd.DataFrame(short_url).url.tolist()

In [50]:
domain_list[:10]

[['edition.cnn.com'],
 ['stanford.edu'],
 ['bandedessinee.blog.lemonde.fr'],
 ['link.springer.com'],
 ['www.sankeisquare.com'],
 ['gifthefeeling.com'],
 ['archive.org'],
 ['www.nature.com'],
 ['kite.com'],
 ['www.penguin.co.uk']]

In [ ]:
x = 1166
for c, i in enumerate(dfz.urls[x:]):
    print(c+x, i)
    if (c + x + 1) % 100 == 0:
        print(f"{(c+1)} / {len(dfz.urls)}")
    for d in i:
        dd = get_domain(d)
        e = expand_link(d)
        de = get_domain(e)
        if not (de == dd):
            short_url.append(
                {"url":d, "short_url": dd, "long_url": de, "is_news": (de in news_domains)}
            )
    with open('short_url.json', 'w') as file:
        file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[3573:3574]

In [ ]:


# x = 3574
# for c, i in enumerate(dfz.urls[x:]):
#     print(c+x, i)
#     if (c + x + 1) % 100 == 0:
#         print(f"{(c+1)} / {len(dfz.urls)}")
#     for d in i:
#         dd = get_domain(d)
#         e =     {
        "url": "https://www.fast.ai/2020/08/21/fastai2-launch/",
        "title": "fast.ai releases new deep learning course, four libraries, and 600-page book \u00b7 fast.ai",
        "short_url": "fast.ai",
        "long_url": "fast.ai",
        "is_news": false,
        "diff_url": false
    },(d)
#         de = get_domain(e)
#         if not (de == dd):
#             short_url.append(
#                 {"url":d, "short_url": dd, "long_url": de, "is_news": (de in news_domains)}
#             )
#     with open('short_url.json', 'w') as file:
#         file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[3586:3587]

In [ ]:
def expand_link(url):
    if url in ["https://", "http://"]:
        return url
    else:
        try:
            session = requests.Session()
            resp = session.head(url, allow_redirects=True)
            long_url = resp.url
            return long_url
        except Exception as ex:
            return url

In [ ]:
x = 3587
for c, i in enumerate(dfz.urls[x:]):
    print(c+x, i)
    if (c + x + 1) % 100 == 0:
        print(f"{(c+1)} / {len(dfz.urls)}")
    for d in i:
        dd = get_domain(d)
        e = expand_link(d)
        de = get_domain(e)
        if not (de == dd):
            short_url.append(
                {"url":d, "short_url": dd, "long_url": de, "is_news": (de in news_domains)}
            )
    with open('short_url.json', 'w') as file:
        file.write(json.dumps(short_url, indent=4))

In [ ]:
dfz.urls[5652:5653]

In [ ]:
bitx = 5653
for c, i in enumerate(dfz.urls[x:]):
    print(c+x, i)
    if (c+x+1) % 100 == 0:
        print(f"{(c+x+1)} / {len(dfz.urls)}")
    for d in i:
        dd = get_domain(d)
        e = expand_link(d)
        de = get_domain(e)
        if not (de == dd):
            short_url.append({"short_url": dd, "long_url": de, "is_news": (de in news_domains)})
    with open('short_url', 'w') as file:
        file.write(json.dumps(short_url, indent=4))